-----------------------
## 토픽모델링에 사용할 네이버 뉴스 데이터 가져오기
- 1. 네이버 오픈 API로 기사 URL 크롤링
- 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [2]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 30

client_id = "frBMA0JHYp1zATqntdUJ"
client_secret = "CzMu_Uce6r"

encText = urllib.parse.quote("롯데 자이언츠")

for idx in range(page_count):
    # json 결과
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1)
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
    #    response_body = response.read()
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())
    #    print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [3]:
print(news_data)
# print(len(news_data))
# print(news_data[0])
# print(news_data[0]['items'])
# print(len(news_data[0]['items']))
# print(news_data[0]['items'][2])
# print(news_data[0]['items'][7]['link'])

SG 추신수가 11일 오후 <b>롯데자이언츠</b>와 SSG랜더스의 연습경기가 열린 부산 사직야구장에 도착해 경기 후 선수단과 상견례를 갖고 있다. 김원형 감독이 유니폼을 입고 그라운드에 나온 추신수를... ', 'pubDate': 'Thu, 11 Mar 2021 16:39:00 +0900'}]}, {'lastBuildDate': 'Fri, 12 Mar 2021 15:35:39 +0900', 'total': 432524, 'start': 251, 'display': 10, 'items': [{'title': 'SSG 선수단과 상견례 갖는 추신수[포토]', 'originallink': 'http://www.xportsnews.com/?ac=article_view&entry_id=1399432', 'link': 'https://sports.news.naver.com/news.nhn?oid=311&aid=0001274058', 'description': '11일 오후 부산 동래구 사직야구장 열린 SSG 랜더스와 <b>롯데 자이언츠</b>의 연습경기 후 SSG 추신수와 선수단의 상견례가 진행됐다. SSG 추신수가 선수단과 상견례를 갖고 있다.', 'pubDate': 'Thu, 11 Mar 2021 16:39:00 +0900'}, {'title': "김원형 감독 '추신수 합류, 이제는 완전체'[포토]", 'originallink': 'http://www.xportsnews.com/?ac=article_view&entry_id=1399430', 'link': 'https://sports.news.naver.com/news.nhn?oid=311&aid=0001274057', 'description': '11일 오후 부산 동래구 사직야구장 열린 SSG 랜더스와 <b>롯데 자이언츠</b>의 연습경기 후 SSG 추신수와 선수단의 상견례가 진행됐다. SSG 김원형 감독이 선수단에게 추신수를 소개하고 있다.', 'pubDate': 'Thu, 11 Mar 2021 16:39:00 +

In [4]:
yna_news_link = []

for page in news_data:
    #print(page)
    page_news_link = []
    
    for item in page['items']:
#         print(item)
        temp_link = item['originallink']
#         print(temp_link)
        if "yna" in temp_link:
            page_news_link.append(temp_link)
    
    yna_news_link.append(page_news_link)
        

# 사이트 링크만 가져오기
for page in yna_news_link:
    for link in page:
        print(link)

http://yna.kr/PYH20210312030400007?did=1196m
http://yna.kr/AKR20210312046800007?did=1195m
http://yna.kr/AKR20210311160351007?did=1195m
http://yna.kr/AKR20210311166700007?did=1195m
http://yna.kr/AKR20210311160300007?did=1195m


In [5]:
yna_news_link = [v for v in yna_news_link if v]

### 가상 크롬드라이버 다운로드
- 크롬 드라이브 다운로드 : https://chromedriver.chromium.org/downloads
- 윈도우 10의 경우 chromedriver.exe

In [6]:
from selenium import webdriver

driver = webdriver.Chrome('./driver/chromedriver')

### 라이브러리 불러오기

In [7]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [8]:
yna_news_title = []
yna_news_content = []


for n in tqdm_notebook(range(len(yna_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(yna_news_link[n]))):        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(yna_news_link[n][idx])
            print(yna_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######        
        title = None
        
        try:
            item = soup.find('div', class_="content03")
            title = item.find('h1', class_="tit").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "story-news article"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    yna_news_title.append(news_page_title)
    yna_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(yna_news_title[0])
print("==================================")
print(yna_news_content[0])

<ipython-input-8-d34c4741f7df>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(yna_news_link))):


  0%|          | 0/4 [00:00<?, ?it/s]

<ipython-input-8-d34c4741f7df>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(yna_news_link[n]))):


  0%|          | 0/2 [00:00<?, ?it/s]

http://yna.kr/PYH20210312030400007?did=1196m
http://yna.kr/AKR20210312046800007?did=1195m


...

http://yna.kr/AKR20210311160351007?did=1195m


  0%|          | 0/1 [00:00<?, ?it/s]

http://yna.kr/AKR20210311166700007?did=1195m


  0%|          | 0/1 [00:00<?, ?it/s]

http://yna.kr/AKR20210311160300007?did=1195m
['OUTLINK', '롯데 한동희, 경남고에 1천300만원 상당 야구용품 선물']
['OUTLINK', '              하남직 기자   기자 페이지                     모교 경남고에 야구용품을 선물한 한동희(왼쪽)   [롯데 자이언츠 제공. 재판매 및 DB금지]          (서울=연합뉴스) 하남직 기자 = 프로야구 롯데 자이언츠 거포 내야수 한동희(22)가 모교인 경남고에 1천300만원 상당의 야구용품을 선물했다.     롯데 구단은 12일 "한동희가 스프링캠프 휴식일인 10일 경남고를 방문해 야구용품을 전달했다"며 "모교 후배들과 대화를 나누며 조언을 하고, 사인볼도 증정했다"고 전했다.     한동희는 "좋은 선수가 되는 것도 중요하지만, 앞서 사회공헌활동을 해왔던 선배들을 따라 \'좋은 사람이 되고 싶다\'라고 생각했다"며 "부족하지만 이번에 전달한 야구용품이 모교 후배들에게 조금이나마 도움이 됐으면 좋겠다"라고 밝혔다.     광고            jiks79@yna.co.kr     <저작권자(c) 연합뉴스,   무단 전재-재배포 금지>   2021/03/12 09:45 송고    ']


In [46]:
del yna_news_title[0][0]
del yna_news_content[0][0]
print(yna_news_title[0])

['롯데 한동희, 경남고에 1천300만원 상당 야구용품 선물']
/Users/lisemara/miniforge3/envs/tf/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
print(yna_news_content[0])

['              하남직 기자   기자 페이지                     모교 경남고에 야구용품을 선물한 한동희(왼쪽)   [롯데 자이언츠 제공. 재판매 및 DB금지]          (서울=연합뉴스) 하남직 기자 = 프로야구 롯데 자이언츠 거포 내야수 한동희(22)가 모교인 경남고에 1천300만원 상당의 야구용품을 선물했다.     롯데 구단은 12일 "한동희가 스프링캠프 휴식일인 10일 경남고를 방문해 야구용품을 전달했다"며 "모교 후배들과 대화를 나누며 조언을 하고, 사인볼도 증정했다"고 전했다.     한동희는 "좋은 선수가 되는 것도 중요하지만, 앞서 사회공헌활동을 해왔던 선배들을 따라 \'좋은 사람이 되고 싶다\'라고 생각했다"며 "부족하지만 이번에 전달한 야구용품이 모교 후배들에게 조금이나마 도움이 됐으면 좋겠다"라고 밝혔다.     광고            jiks79@yna.co.kr     <저작권자(c) 연합뉴스,   무단 전재-재배포 금지>   2021/03/12 09:45 송고    ']
/Users/lisemara/miniforge3/envs/tf/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
print(len(yna_news_title))
print(len(yna_news_content))

4
4


In [48]:
with open("yna_news_title.pk", "wb") as f:
    pickle.dump(yna_news_title, f)
    
with open("yna_news_content.pk", "wb") as f:
    pickle.dump(yna_news_content, f)

/Users/lisemara/miniforge3/envs/tf/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-----------------------------------------------
## gensim으로 네이버 뉴스를 이용한 토픽 모델링 
- 1. 토픽 모델링을 적용하기 위해 텍스트를 전처리
- 2. gensim을 사용한 토픽 모델링 

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [49]:
from tqdm import tqdm_notebook # progress bar
# from konlpy.tag import Okt #Okt 등 형태소 분석기 불러오기
from konlpy.tag import Mecab #Okt 등 형태소 분석기 불러오기
import numpy as np
import string # 특수문자
import re
import warnings # 경고 알림 제거
import pickle
from gensim import corpora
from gensim import models
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) 

/Users/lisemara/miniforge3/envs/tf/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 2. 텍스트 전처리 함수 만들기

In [50]:
def read_documents(input_file_name):    
    corpus = []    
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", str(doc))

    return docs

def define_stopwords(path):    
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation:
        SW.add(i)
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가    
    with open(path, encoding='cp949') as f:
        for word in f:
            SW.add(word)
    return SW

def text_tokenizing(corpus, tokenizer):    
    mecab = Mecab()
    token_corpus = []    

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]                
            token_corpus.append(token_text)
            
    elif tokenized == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)        

    return token_corpus

In [51]:
input_file_name = "yna_news_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("data/stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word"

Preprocessing:   0%|          | 0/4 [00:00<?, ?it/s]

In [67]:
print(tokenized_text[0])

['남직', '기자', '기자', '페이지', '모교', '남고', '야구', '용품', '선물', '한동희', '왼쪽', '롯데', '자이언츠', '제공', '재판매', '금지', '서울', '연합뉴스', '남직', '기자', '프로야구', '롯데', '자이언츠', '거포', '내야수', '한동희', '모교', '남고', '상당', '야구', '용품', '선물', '롯데', '구단', '한동희', '스프링', '캠프', '휴식', '남고', '방문', '야구', '용품', '전달', '모교', '후배', '대화', '조언', '증정', '한동희', '선수', '중요', '사회', '공헌', '활동', '선배', '사람', '생각', '부족', '이번', '전달', '야구', '용품', '모교', '후배', '조금', '도움', '광고', '저작권자', '연합뉴스', '무단', '전재', '배포', '금지', '송고']


In [53]:
import nltk
print(len(tokenized_text))

4


In [57]:
from pprint import pprint
text = nltk.Text(tokenized_text, name='yna_news')
print(len(set(tokenized_text)))
pprint(text.vocab().most_common(10))

TypeError: unhashable type: 'list'

In [31]:
import matplotlib.pyplot as plt
import platform
from matplotlib import font_manager, rc
%matplotlib inline

path = '/Library/Fonts/NanumGothic.ttf'
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')

plt.figure(figsize=(12, 7))
text.plot(50)

/Users/lisemara/miniforge3/envs/tf/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'list' object has no attribute 'plot'

### 3. 토픽 모델링에 사용할 행렬 생성

### 1) 문서-단어 행렬(DTM, document-term matrix) 생성

In [19]:
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)

# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [20]:
print(dictionary)

Dictionary(209 unique tokens: ['거포', '공헌', '광고', '구단', '금지']...)


In [21]:
corpus[0][:5]

[]

### 2) TFIDF 문서-단어 행렬 생성

In [22]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[]

### 4.  LDA 모델 생성

In [23]:
model = models.ldamodel.LdaModel(corpus, num_topics=4, id2word=dictionary)

In [24]:
model.show_topic(0, 10)

[('추신수', 0.04327199),
 ('이태양', 0.029701898),
 ('부산', 0.023130702),
 ('기자', 0.02045096),
 ('번호', 0.019070456),
 ('연합뉴스', 0.018701939),
 ('양보', 0.018098399),
 ('롯데', 0.017474854),
 ('시계', 0.016502688),
 ('선물', 0.014068911)]

In [25]:
model.show_topic(1, 10)

[('롯데', 0.025571618),
 ('야구', 0.023248201),
 ('기자', 0.02100569),
 ('한동희', 0.019122478),
 ('용품', 0.018341662),
 ('모교', 0.017947491),
 ('추신수', 0.0167553),
 ('연합뉴스', 0.01651332),
 ('선물', 0.015620277),
 ('금지', 0.015171738)]

In [26]:
model.show_topic(2, 10)

[('추신수', 0.044528835),
 ('이태양', 0.023159178),
 ('선물', 0.023054125),
 ('시계', 0.021586407),
 ('부산', 0.021033982),
 ('번호', 0.019583408),
 ('기자', 0.019409506),
 ('양보', 0.016823657),
 ('선수단', 0.016457172),
 ('연합뉴스', 0.01615396)]

In [27]:
model.show_topic(3, 10)

[('추신수', 0.038895514),
 ('이태양', 0.029602436),
 ('부산', 0.028829768),
 ('번호', 0.027206928),
 ('기자', 0.026603803),
 ('선물', 0.023000984),
 ('연합뉴스', 0.021927537),
 ('시계', 0.021698775),
 ('롯데', 0.019426199),
 ('양보', 0.016515195)]

### 5. 토픽 모델링을 추가하여 코드 완성하기

In [28]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 3 # 토픽의 수

NUM_TOPIC_WORDS = 30 # 키워드 수

def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]        
    return corpus, dictionary

def print_topic_words(model):
    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))            
        print("\n")

In [29]:
# 1) document-term matrix 생성
corpus, dictionary = build_doc_term_mat(tokenized_text)

# 2) LDA 모델 생성
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")

# 3) 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	추신수	0.042622774839401245
	이태양	0.028199099004268646
	부산	0.025736991316080093
	기자	0.025426819920539856
	선물	0.022111281752586365
	연합뉴스	0.022104091942310333
	롯데	0.021208619698882103
	번호	0.021184829995036125
	시계	0.01958382874727249
	양보	0.015686415135860443
	경기	0.013512348756194115
	선수단	0.013182474300265312
	연습	0.01303454302251339
	구장	0.012746717780828476
	사직	0.011957954615354538
	전달	0.011945323087275028
	금지	0.01129697822034359
	프로야구	0.011033759452402592
	해당	0.010975095443427563
	선수	0.010630658827722073
	야구	0.00992579571902752
	강덕	0.009886115789413452
	리그	0.009764032438397408
	오후	0.009531479328870773
	고가	0.009417872875928879
	상견례	0.009195533581078053
	이후	0.008874778635799885
	구단	0.008585216477513313
	송승준	0.00854345690459013
	생활	0.007812442258000374


Topic ID: 1
	추신수	0.026342032477259636
	기자	0.023174861446022987
	이태양	0.02172687277197838
	부산	0.017615241929888725
	롯데	0.017392920330166817
	번호	0.016635103151202202
	연합뉴스	0.016581

### 6. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [ ]:
#!pip install pyLDAvis

In [30]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.009906 -0.002002       1        1  87.844249
1      0.009618 -0.002188       2        1  11.785311
2      0.000288  0.004190       3        1   0.370440, topic_info=     Term       Freq      Total Category  logprob  loglift
115   추신수  26.000000  26.000000  Default  30.0000  30.0000
80     부산  15.000000  15.000000  Default  29.0000  29.0000
100   이태양  17.000000  17.000000  Default  28.0000  28.0000
79     번호  13.000000  13.000000  Default  27.0000  27.0000
22     선물  13.000000  13.000000  Default  26.0000  26.0000
..    ...        ...        ...      ...      ...      ...
44   프로야구   0.017322   6.829891   Topic3  -4.9190  -0.3788
96     연습   0.017815   7.987036   Topic3  -4.8909  -0.5073
24     선수   0.016840   6.954255   Topic3  -4.9472  -0.4251
4      금지   0.016124   7.259191   Topic3  -4.9907  -0.5115
35     전달   0.015980   7.545404   Topic3  -4.9996  -0.5591

[206 rows x 6 columns], token_table=      Topic      Freq Term
term                      
50        1  1.012129   강덕
0         1  0.948028   거포
52        1  0.838772   격리
54        1  0.961043   경기
54        2  0.120130   경기
...     ...       ...  ...
128       2  0.144133   해당
46        1  0.542351   활동
47        1  0.915650   후배
47        2  0.228913   후배
48        1  0.952905   휴식

[134 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])